# Unified Data Setup, Pre-Modeling

By: Jared Berry

In [1]:
# Import necessary libraries for data preparation/EDA
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

### Assemble master data set

In [2]:
# Connect to PostgresDB and pull in datasets
engine = create_engine("postgresql://postgres:dfdk#418!@@34.74.173.183/postgres")

In [3]:
# Yahoo! Finance
yahoo=pd.read_sql_query('select * from stock_price', con=engine)
print("Yahoo! Finance features:")
print(yahoo.columns.tolist())

Yahoo! Finance features:
['sno', 'date_of_transaction', 'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Symbol', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed']


In [4]:
# SimFin Fundamentals
simfindaily=pd.read_sql_query('select * from daily_simfin', con=engine)
print("SimFin features:")
print(simfindaily.columns.tolist())

SimFin features:
['date', 'ticker', 'qtr_yr', 'cash_from_financing_activities', 'cash_from_investing_activities', 'cash_from_operating_activities', 'cash_from_repayment_of_debt', 'cash_from_repurchase_of_equity', 'change_in_fixed_assets_intangibles', 'change_in_working_capital', 'depreciation_amortization_x', 'net_cash_before_disc_operations_and_fx', 'net_cash_before_fx', 'net_changes_in_cash', 'net_incomestarting_line', 'non_cash_items', 'other_financing_activities', 'gross_profit', 'income_loss_from_continuing_operations', 'income_loss_including_minority_interest', 'income_tax_expense_benefit_net', 'net_income_y', 'net_income_available_to_common_shareholders', 'non_operating_income_loss', 'operating_expenses', 'operating_income_loss', 'pretax_income_loss', 'pretax_income_loss_adjusted', 'revenue', 'accounts_notes_receivable', 'cash_cash_equivalents', 'cash_cash_equivalents_short_term_investments', 'equity_before_minority_interest', 'long_term_debt', 'other_equity', 'other_long_term_a

In [5]:
# Derived momentum features
momentum=pd.read_sql_query('select * from momentum_features', con=engine)
print("Derived features")
print(momentum.columns.tolist())

Derived features
['Symbol', 'Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Pct_Change_Daily', 'Pct_Change_Monthly', 'Pct_Change_Yearly', 'RSI', 'Volatility', 'Yearly_Return_Rank', 'Monthly_Return_Rank', 'Pct_Change_Class', 'Rolling_Yearly_Mean_Positive_Days', 'Rolling_Monthly_Mean_Positive_Days', 'Rolling_Monthly_Mean_Price', 'Rolling_Yearly_Mean_Price', 'Momentum_Quality_Monthly', 'Momentum_Quality_Yearly', 'SPY_Trailing_Month_Return']


In [6]:
# S&P 500 index
snp = pd.read_sql_query('select * from spy_stock_price', con = engine)
print("S&P 500")
print(snp.columns.tolist())

S&P 500
['Sno', 'date_of_transaction', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'symbol', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed']


In [7]:
# Some quick fixes on keys
simfindaily['date_of_transaction'] = simfindaily['date']
simfindaily.drop('date', axis=1, inplace=True) 

yahoo['ticker'] = yahoo['Symbol']
yahoo.drop('Symbol', axis=1, inplace=True)

momentum['ticker'] = momentum['Symbol']
momentum['date_of_transaction'] = momentum['Date']
momentum.drop(['Symbol', 'Date', 'High', 'Low', 
               'Open', 'Close', 'Volume', 'AdjClose'], 
              axis=1, inplace=True)

snp['snp500_close'] = snp['Adj Close']
snp['snp500_open'] = snp['Open']
snp = snp[['date_of_transaction', 'snp500_close', 'snp500_open']]

In [8]:
# Merge
df = pd.merge(yahoo, momentum, on=['ticker', 'date_of_transaction'])
df = pd.merge(df, simfindaily, how='left', on=['ticker', 'date_of_transaction'])

In [9]:
df = df.sort_values(['ticker','date_of_transaction']).reset_index(drop = True)
df.head()

,sno,date_of_transaction,High,Low,Open,Close,Volume,AdjClose,Year,Month,...,total_assets,total_current_assets,total_current_liabilities,total_equity,total_liabilities,total_liabilities_equity,total_noncurrent_assets,total_noncurrent_liabilities,common_outstanding_basic,common_outstanding_diluted
0,22334,2011-01-03,30.143061,29.620888,29.728184,29.957081,4994000.0,27.591616,2011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22335,2011-01-04,30.114449,29.456366,30.035765,29.678112,5017200.0,27.334681,2011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22336,2011-01-05,29.849785,29.327610,29.513592,29.613733,4519000.0,27.275387,2011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22337,2011-01-06,29.928469,29.477825,29.592276,29.670958,4699000.0,27.328091,2011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22338,2011-01-07,29.899857,29.356224,29.699572,29.771101,3810900.0,27.420322,2011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Pull out the tickers
tickers = df['ticker'].unique().tolist()

### Combined data set feature engineering

In [11]:
# Construct some aggregate financial ratios from the SimFin data
df['eps'] = df['net_income_y'] / df['common_outstanding_basic']
df['pe_ratio'] = df['AdjClose'] / df['eps']
df['debt_ratio'] = df['total_liabilities'] / df['total_equity']
df['debt_to_equity'] = df['total_liabilities'] / df['total_equity']
df['roa'] = df['net_income_y'] / df['total_assets']

In [12]:
# Construct some additional ticker-level returns features
df['open_l1'] = df.groupby('ticker')['Open'].shift(1)
df['open_l5'] = df.groupby('ticker')['Open'].shift(5)
df['open_l10'] = df.groupby('ticker')['Open'].shift(10)

df['return_prev1_open_raw'] = 100*(df['Open'] - df['open_l1'])/df['open_l1']
df['return_prev5_open_raw'] = 100*(df['Open'] - df['open_l5'])/df['open_l5']
df['return_prev10_open_raw'] = 100*(df['Open'] - df['open_l10'])/df['open_l10']

df['close_l1'] = df.groupby('ticker')['AdjClose'].shift(1)
df['close_l5'] = df.groupby('ticker')['AdjClose'].shift(5)
df['close_l10'] = df.groupby('ticker')['AdjClose'].shift(10)

df['return_prev1_close_raw'] = 100*(df['AdjClose'] - df['close_l1'])/df['close_l1']
df['return_prev5_close_raw'] = 100*(df['AdjClose'] - df['close_l5'])/df['close_l5']
df['return_prev10_close_raw'] = 100*(df['AdjClose'] - df['close_l10'])/df['close_l10']

In [14]:
# Compute market betas
betas = np.empty(df.shape[0])
for t in tickers:
    idx = df['ticker'].loc[df['ticker'] == t].index.tolist()
    x_t = df[['date_of_transaction', 'AdjClose']].iloc[idx]

    x_t = pd.merge(x_t, snp, on='date_of_transaction').sort_values('date_of_transaction')

    market_return = np.array(x_t['snp500_close'].tolist())
    asset_return = np.array(x_t['AdjClose'].tolist())

    beta_vector = np.empty(len(asset_return)) * np.nan
    i = 21
    while i < len(beta_vector):
        beta_vector[i] = np.cov(market_return[:(i-1)], asset_return[:(i-1)])[0,1] / np.var(market_return[:(i-1)])
        i += 1
        
    betas[idx] = beta_vector
    
df['beta'] = betas

In [15]:
# Features to smooth
to_smooth = ['High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Pct_Change_Daily',
            'Pct_Change_Monthly', 'Pct_Change_Yearly', 'RSI', 'Volatility',
            'Yearly_Return_Rank', 'Monthly_Return_Rank', 'Pct_Change_Class',
            'Rolling_Yearly_Mean_Positive_Days', 'Rolling_Monthly_Mean_Positive_Days', 
            'Rolling_Monthly_Mean_Price', 'Rolling_Yearly_Mean_Price',
            'open_l1', 'open_l5', 'open_l10', 'close_l1', 'close_l5', 'close_l10',
            'return_prev1_open_raw', 'return_prev5_open_raw', 'return_prev10_open_raw',
            'return_prev1_close_raw', 'return_prev5_close_raw', 'return_prev10_close_raw',
            'pe_ratio', 'debt_ratio', 'debt_to_equity', 'roa', 'Momentum_Quality_Monthly', 
             'Momentum_Quality_Yearly', 'SPY_Trailing_Month_Return'
            ]

In [ ]:
# Create smoothed variants of specified features
for feature in to_smooth:
    x_to_smooth = np.array(df[feature].tolist())
    col = feature + "_smoothed"
    for t in tickers:
        idx = df['ticker'].loc[df['ticker'] == t].index.tolist()
        x_t = np.array(x_to_smooth[idx].tolist())

        # Compute EMA smoothing of target within ticker
        EMA = 0
        gamma_ = 0.5
        for ti in range(len(x_t)):
            EMA = gamma_*x_t[ti] + (1-gamma_)*EMA
            x_t[ti] = EMA

        x_to_smooth[idx] = x_t
    df[col] = x_to_smooth

In [16]:
# Hash the ticker to create a categorical feature
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features = len(tickers), input_type = 'string')
f = h.transform(df['ticker'])
ticker_features = f.toarray()

In [17]:
# Remove the quarter of pre-SimFin data
train = df[df['date_of_transaction'] >= '2011-03-31'].reset_index(drop=True)

### Target generation

We construct a series of potential targets to frame our classification exercise. Return variants are binarized in modeling frameworks, allowing for the application of possible smoothing frameworks.

The n-day ahead return is calculated as:
$$target_{t,i} = \frac{AdjClose_{t+n,i} - AdjClose_{t,i}}{AdjClose_{t,i}}$$

The average, returns for all periods within the next n-days (relative to today) is calculated as:
$$target_{t,i} = (\frac{1}{n})\sum_{k=1}^n \frac{AdjClose_{t+k,i} - AdjClose_{t,i}}{AdjClose_{t,i}}$$

The q-day moving average of n-day ahead raw returns, relative to today is calculated as:
$$target_{t,i,q} = \frac{AdjClose_{t+n,i} - AdjClose_{t,i}}{AdjClose_{t,i}}, MA(q)$$

Relative variants are relative to the S&P 500 over the same n-day horizon.
'Up' and 'Rank' variants are calculated without computing returns over the same n-day horizon, and binarized.

In [20]:
# At the ticker level, lead the AdjClose column by n-trading days
target_gen = train[['ticker', 'date_of_transaction', 'AdjClose', 'Monthly_Return_Rank', 'beta']]
target_gen = pd.merge(target_gen, snp, on='date_of_transaction')

In [21]:
# Loop over specified horizons to generate a number of possible targets
horizons = [1,5,10,21]
rank_threshold = 100
target_dict = defaultdict(list)
for h in horizons:
    n = h # n-day ahead return
    q = h # q-day window
    
    # At the ticker level, lead the AdjClose column n-trading days
    AdjClose_ahead = target_gen.groupby('ticker')['AdjClose'].shift(-n)
    AdjClose_ahead.name = 'AdjClose_ahead'
    
    snp_ahead = target_gen.groupby('ticker')['snp500_close'].shift(-n)
    snp_ahead.name = 'snp_ahead'
    
    # Raw returns
    target_return = np.array(100*((AdjClose_ahead - target_gen['AdjClose'])/target_gen['AdjClose']))
    
    # Market residualized returns
    target_return_res = target_return - np.array(target_gen['beta'].tolist())*target_return
    
    # Computing all of the returns for the next 21 days (month) relative to today
    aheads = []
    for i in range(0,n+1):
        AdjClose_ahead_i = target_gen.groupby('ticker')['AdjClose'].shift(-i)
        aheads.append(np.array(100*((AdjClose_ahead_i - target_gen['AdjClose'])/target_gen['AdjClose'])))
    
    # Composite, average returns
    target_composite = np.array(pd.DataFrame(aheads).mean(axis=0, skipna=False).tolist())
    
    # q-day moving average of n-day ahead returns, where n=q
    target_gen['returns_ahead'] = 100*((AdjClose_ahead - target_gen['AdjClose'])/target_gen['AdjClose'])
    target_average = np.array(target_gen.groupby('ticker')['returns_ahead'].rolling(q).mean())
    
    # Rank target, binarized
    target_rank = target_gen.groupby('ticker')['Monthly_Return_Rank'].shift(-n)
    target_rank = np.where(np.isnan(target_rank), np.nan,
                  np.where(target_rank < rank_threshold, 1, 0))
    target_rank = target_rank.tolist()
    
    # Simple 'up' target, relative to today
    target_up = np.where(np.isnan(AdjClose_ahead), np.nan,
                np.where(AdjClose_ahead > target_gen['AdjClose'], 1, 0))
    target_up = target_up.tolist()
    
    # Returns, relative to the S&P 500
    snp_return = np.array(100*((snp_ahead - target_gen['snp500_close'])/target_gen['snp500_close']))
    target_rel_return = target_return - snp_return
    
    # Generate keys based on horizon
    return_key = "target_{}_return".format(n)
    return_res_key = "target_{}_return_res".format(n)
    composite_key = "target_{}_composite".format(n)
    average_key = "target_{}_average".format(n)
    rank_key = "target_{}_rank".format(n)
    up_key = "target_{}_up".format(n)
    rel_return_key = "target_{}_rel_return".format(n)
    
    # Store
    target_dict[return_key] = target_return
    target_dict[return_res_key] = target_return_res
    target_dict[composite_key] = target_composite
    target_dict[average_key] = target_average
    target_dict[rank_key] = target_rank
    target_dict[up_key] = target_up
    target_dict[rel_return_key] = target_rel_return

In [22]:
# Add features to dictionary prior to export
target_dict['features'] = train
target_dict['ticker_features'] = ticker_features

In [23]:
# Export
outpath = "model_dictionary.pickle"
with open(outpath, 'wb') as f:
    pickle.dump(target_dict, f)